In [1]:
def main():
    import jax.experimental.sparse as jesp
    import networkx as nx
    import scipy.sparse as sp
    import jax.numpy as jnp
    import numpy as np
    import matplotlib.pyplot as plt
    import msprime as msp
    import networkx as nx
    from parameters_EEMS_old import MCMCParams
    import random

    def setup_graph(
        n_rows=8,
        n_columns=12,
        barrier_startpt=2.5,
        barrier_endpt=8.5,
        anisotropy_scaler=1.0,
        barrier_w=0.1,
        corridor_w=0.5,
        n_samples_per_node=10,
        barrier_prob=0.1,
        corridor_left_prob=1,
        corridor_right_prob=1,
        sample_prob=1.0,
    ):
        demo = msp.Demography()
        testing_pairs = []
        test_q = []
        test_m = []
        pair_order = []
        graph = nx.generators.lattice.triangular_lattice_graph(
            n_rows - 1, 2 * n_columns - 2, with_positions=True
        )
        graph = nx.convert_node_labels_to_integers(graph)
        pos_dict = nx.get_node_attributes(graph, "pos")
        for i in graph.nodes:
            pop_name = f"P{i}"
            q = random.uniform(0, 2)
            test_q.append(q)
            demo.add_population(initial_size= (2*Ne) / q, name=pop_name)
            testing_pairs.append((i, i))

            # node position
            x, y = graph.nodes[i]["pos"]

            if x <= barrier_startpt:
                graph.nodes[i]["sample_size"] = 2 * np.random.binomial(
                    n_samples_per_node, corridor_left_prob
                )
            elif x >= barrier_endpt:
                graph.nodes[i]["sample_size"] = 2 * np.random.binomial(
                    n_samples_per_node, corridor_right_prob
                )
            else:
                graph.nodes[i]["sample_size"] = 2 * np.random.binomial(
                    n_samples_per_node, barrier_prob
                )

            # sample a node or not
            graph.nodes[i]["sample_size"] = graph.nodes[i][
                "sample_size"
            ] * np.random.binomial(1, sample_prob)

        # assign edge weights
        for i, j in graph.edges():
            neighbor1 = f"P{i}"
            neighbor2 = f"P{j}"
            pair_order.append((i, j))
            
            x = np.mean([graph.nodes[i]["pos"][0], graph.nodes[j]["pos"][0]])
            y = np.mean([graph.nodes[i]["pos"][1], graph.nodes[j]["pos"][1]])
            if x <= barrier_startpt:
                graph[i][j]["w"] = corridor_w
                demo.set_symmetric_migration_rate(
                    populations=(neighbor1,neighbor2), rate=corridor_w / (4 * Ne))
                test_m.append(corridor_w)
            elif x >= barrier_endpt:
                graph[i][j]["w"] = corridor_w
                demo.set_symmetric_migration_rate(
                    populations=(neighbor1,neighbor2), rate=corridor_w / (4 * Ne))
                test_m.append(corridor_w)
            else:
                graph[i][j]["w"] = barrier_w
                demo.set_symmetric_migration_rate(
                    populations=(neighbor1,neighbor2), rate=barrier_w / (4 * Ne))
                test_m.append(barrier_w)

            # if horizontal edge
            if graph.nodes[i]["pos"][1] == graph.nodes[j]["pos"][1]:
                graph[i][j]["w"] = anisotropy_scaler * graph[i][j]["w"]

        return graph, demo, test_m, test_q, testing_pairs, pair_order, graph.number_of_nodes()

    # Example usage
    Ne = 1e4           # Effective population size

    # Generate the demography and graph for the 20x10 grid
    nrows = 2
    ncols = 3
    basic_graph, demo, test_m, test_q, testing_pairs, pair_order, total_nodes = setup_graph(n_rows = nrows, n_columns = ncols, barrier_startpt=ncols/3, barrier_endpt=2*ncols/3, barrier_w=0.01, corridor_w=0.1)

    import demes
    cmd = demes.ms.to_ms(demo.to_demes(), N0=1e4, samples=[2] * total_nodes)

    L = 1e7
    theta = 4 * 1e-8 * Ne * L
    r = 4 * 1e-8 * Ne * L

    # you use os if you don't care to store the output from the terminal
    # all I need is the output.txt which I can create and read anyways so I use os
    import os
    command = f"/home/jkliang/EEMS_old_code_v.1.27/scrm-1.7.4/scrm {2*total_nodes} 1 -t {theta} -r {r} {L} --transpose-segsites -SC abs -p 14 -oSFS {cmd} > /home/jkliang/EEMS_old_code_v.1.27/output.txt"
    os.system(command)
    from phlash.sim import _parse_scrm
    from phlash.data import VcfContig
    with open("/home/jkliang/EEMS_old_code_v.1.27/output.txt", "r") as scrm_out:
        vcf = _parse_scrm(scrm_out, chrom_name="chr1")

    open("/home/jkliang/EEMS_old_code_v.1.27/output.vcf", "wt").write(vcf)
    command = f"bcftools view output.vcf -o /home/jkliang/EEMS_old_code_v.1.27/output.bcf"
    os.system(command)
    command = f"bcftools index /home/jkliang/EEMS_old_code_v.1.27/output.bcf"
    os.system(command)
    samples = [f"sample{i}" for i in range(total_nodes)]
    TreeSeq = VcfContig("/home/jkliang/EEMS_old_code_v.1.27/output.bcf", samples=samples, contig = "chr1", interval = (0, 1e7))
###
    print(testing_pairs)

    rows = 20
    cols = 20

    num_nodes = rows*cols
    G = nx.grid_2d_graph(m=rows, n=cols)
    G = nx.convert_node_labels_to_integers(G)
    num_edges = G.number_of_edges()
    print(num_edges)
    
    tmp = jesp.BCOO.from_scipy_sparse(sp.triu(nx.adjacency_matrix(G, np.arange(num_nodes)).astype(float)))


    # print(TreeSeq.get_data(100))
    # print(TreeSeq.L)

    mutation_rate = 1e-8
    Ne = 1e4
    t1 = 1e-4
    tM = 15.0
    window_size = 100
    theta = 4 * Ne * mutation_rate
    rho = 1.0 * theta
    pat = "14*1+1*2"

    # obtaining upper triangular of adjacency matrix, cannot use np.triu because we are
    # working with a sparse matrix, not a numpy matrix
    initialization = MCMCParams.from_linear(
            pattern=pat,
            rho=rho * window_size,
            t1 = t1, 
            tM = tM,
            BCOO_indices = tmp.indices,
            m = np.ones(num_edges), 
            q = np.ones(num_nodes),
            theta=theta * window_size,
            alpha=0.0,
            beta=0.0,
        )

    from MCMC_Method_EEMS_old_v1_06 import running_MCMC
    from utility_methods_old import chunk_attributes, chunk_map
    num_node_pairs, num_chunks, chunk_length = chunk_attributes(TreeSeq, window_size=100, overlap=500)
    map = chunk_map(testing_pairs, num_chunks)
    print(map)

    particles = running_MCMC(initialization, TreeSeq, testing_pairs, map, num_particles=100)
    print(test_q)
    print(true_m_order)
    
if __name__ == "__main__":
    main()


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
760
[[ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  2  0  2]
 [ 1  7 10 ...  0  0  2]]
20000
(6, 5, 20500)
{(0, 0): Array([0, 1, 2, 3, 4], dtype=int64), (1, 1): Array([5, 6, 7, 8, 9], dtype=int64), (2, 2): Array([10, 11, 12, 13, 14], dtype=int64), (3, 3): Array([15, 16, 17, 18, 19], dtype=int64), (4, 4): Array([20, 21, 22, 23, 24], dtype=int64), (5, 5): Array([25, 26, 27, 28, 29], dtype=int64)}


2025-02-04 08:29:13.097 | INFO     | MCMC_Method_EEMS_old_v1_06:running_MCMC:167 - Loading data


  0%|          | 0.00/120M [00:00<?, ?bp/s]

2025-02-04 08:29:24.207 | DEBUG    | phlash.gpu:initialize_cuda:94 - Compute capability: 86
2025-02-04 08:29:24.208 | INFO     | phlash.gpu:_initialize_devices:370 - Using 1 GPUs


2025-02-04 08:29:31.840901: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 31.66GiB (34000072355 bytes) by rematerialization; only reduced to 64.66GiB (69425555482 bytes), down from 64.66GiB (69425580282 bytes) originally
2025-02-04 08:29:51.543246: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 64.65GiB (rounded to 69418735616)requested by op 
2025-02-04 08:29:51.543352: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] ________________________________________________*****************xxxxxxx*************_______________
E0204 08:29:51.543377 1707635 pjrt_stream_executor_client.cc:3045] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 69418735448 bytes. [tf-allocator-allocation-error='']
Fitting model:   0%|          | 0/2000 [00:26<?, ?it/s]


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 69418735448 bytes.

In [1]:
!nvidia-smi

Thu Jan 30 14:19:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:1E:00.0 Off |                    0 |
|  0%   30C    P8             21W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----